In [2]:
try:
  !rm -rf boptestGymService
except:
  pass
!git clone -b boptest-gym-service https://github.com/ibpsa/project1-boptest-gym.git boptestGymService

Cloning into 'boptestGymService'...


In [3]:
!pip install gymnasium
!pip install stable-baselines3==2.0.0 
!pip install numpy==1.24.4
!pip install swig
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 #this line should change for mac, you can find it on pytorch
!pip install tensorflow
%load_ext tensorboard
import tensorflow as tf
import datetime

  Using cached numpy-1.24.4-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-win_amd64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 1.24.4 which is incompatible.
tensorflow-cpu 2.18.1 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.19.0 which is incompatible.


ERROR: Invalid requirement: '#this': Expected package name at the start of dependency specifier
    #this
    ^


  Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.1.3-cp311-cp311-win_amd64.whl (12.9 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-cpu 2.18.1 requires tensorboard<2.19,>=2.18, but you have tensorboard 2.19.0 which is incompatible.


In [4]:
import json
import sys
import numpy as np
import requests
import gymnasium as gym


In [5]:
sys.path.insert(0,'boptestGymService')
from boptestGymEnv import BoptestGymEnv

In [6]:
# env.stop()

from boptestGymEnv import BoptestGymEnv, NormalizedObservationWrapper, DiscretizedActionWrapper
from stable_baselines3 import DQN

# url for the BOPTEST service
url = 'https://api.boptest.net'

# Decide the state-action space of your test case
env = BoptestGymEnv(
        url                  = url,
        testcase             = 'bestest_hydronic_heat_pump',
        actions              = ['oveHeaPumY_u'],
        observations         = {'time':(0,604800),
                                'reaTZon_y':(280.,310.),
                                'TDryBul':(265,303),
                                'HDirNor':(0,862),
                                'InternalGainsRad[1]':(0,219),
                                'PriceElectricPowerHighlyDynamic':(-0.4,0.4),
                                'LowerSetp[1]':(280.,310.),
                                'UpperSetp[1]':(280.,310.)},
        predictive_period    = 24*3600,
        regressive_period    = 6*3600,
        random_start_time    = True,
        max_episode_length   = 24*3600,
        warmup_period        = 24*3600,
        step_period          = 3600)

# Normalize observations and discretize action space
env = NormalizedObservationWrapper(env)
env = DiscretizedActionWrapper(env,n_bins_act=10)

# Instantiate an RL agent
model = DQN('MlpPolicy', env, verbose=1, gamma=0.99,
            learning_rate=5e-4, batch_size=24, seed=123456,
            buffer_size=365*24, learning_starts=24, train_freq=1)

# Main training loop
model.learn(total_timesteps=10)



C:\Users\irmak\anaconda3\envs\model\Lib\site-packages\gymnasium\spaces\box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


{'tdis_tot': 8.370831655824315,
 'idis_tot': 0,
 'ener_tot': 0.396528447174189,
 'cost_tot': 0.10051996135865693,
 'emis_tot': 0.06622025067808956,
 'pele_tot': 0.021041629787807846,
 'pgas_tot': None,
 'pdih_tot': None,
 'time_rat': 0.0008461354147408897}

In [18]:
# Loop for one episode of experience (one day as set in max_episode_length)
done = False
counter = 0
obs, _ = env.reset()
while not done:
  action, _ = model.predict(obs, deterministic=True)
  print('-------------------------------------------------------------------')
  print(obs)
  print(action)
  print('-------------------------------------------------------------------')
  obs,reward,terminated,truncated,info = env.step(action)
  done = (terminated or truncated)
  counter += 1

# Obtain KPIs for evaluation
env.get_kpis()
print(counter)

-------------------------------------------------------------------
[-0.7869742  -0.09005737 -0.07985431 -0.06905925 -0.07291055 -0.0780375
 -0.08587646 -0.08201092 -0.10207486 -0.10733837 -0.16926813 -0.21374351
 -0.26107144 -0.3034379  -0.3455441  -0.38702232 -0.4483707  -0.5117364
 -0.5341556  -0.51798606 -0.5797055  -0.62153065 -0.5859407  -0.4623863
 -0.36434937 -0.2927776  -0.25445557 -0.19774425 -0.15458602 -0.10202509
 -0.07380599 -0.04224598 -0.03876698 -0.8719969  -0.8125124  -0.8983029
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1.         -1.         -0.989082   -0.710088
 -0.19657052  0.128003    0.4414829   0.7356962   0.7384237   0.52948844
  0.75920093  0.7585585   0.33784628 -0.11469406 -1.         -1.
 -1.          1.          1.          1.          1.          1.
  1.          1.          1.          1.          1.          1.
 -1.         -1.         -1.         -1.         -1.         -1.
 -1.         -1.         -1. 

In [16]:
obs?

Type:        ndarray
String form:
[-0.5337335   0.3738464   0.36892903  0.36483157  0.3561666   0.34435427
           0.324939    0.3121196  <...>  0.54333293  0.54333293  0.54333293  0.54333293  0.54333293  0.54333293
           0.54333293  0.54333293]
Length:      158
File:        c:\users\irmak\anaconda3\envs\model\lib\site-packages\numpy\__init__.py
Docstring:  
ndarray(shape, dtype=float, buffer=None, offset=0,
        strides=None, order=None)

An array object represents a multidimensional, homogeneous array
of fixed-size items.  An associated data-type object describes the
format of each element in the array (its byte-order, how many bytes it
occupies in memory, whether it is an integer, a floating point number,
or something else, etc.)

Arrays should be constructed using `array`, `zeros` or `empty` (refer
to the See Also section below).  The parameters given here refer to
a low-level method (`ndarray(...)`) for instantiating an array.

For more information, refer to the `numpy`

In [20]:
env.stop()